# Install KenLM Library

In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mv kenlm kenlm_tool
!mkdir kenlm_tool/build

--2023-03-17 09:35:12--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  1.66MB/s    in 0.3s    

2023-03-17 09:35:13 (1.66 MB/s) - written to stdout [491888/491888]



In [ ]:
%cd kenlm_tool/build
!cmake ..
!make -j2

/content/kenlm_tool/build
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found suitable version "1.71.0", minimum required is "1.41.0") found components: program_options system thread unit_test_framework 
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11") 
-- Found BZip2: /usr/lib/x86_6

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 553.5 kB 1.8 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.0.0-cp39-cp39-linux_x86_64.whl size=3262184 sha256=3f8fbd239ac6526ccb7c2e2409a2a4b422ae04867a5cf980423b690df4d73c3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-hy1dhomj/wheels/b5/52/c9/af2949d9776846ea81a9cba52a4fe5a81b9ace3b9f2530c3f3
Successfully built kenlm


In [ ]:
!export PATH=$PATH:/root/.local/bin

In [ ]:
!which lmplz

# Download Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!bin/lmplz -o 5 --text '/content/drive/MyDrive/com ling/contest 2/contest 2/train.src.tok' --arpa fivegram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/drive/MyDrive/com ling/contest 2/contest 2/train.src.tok
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 128282272 types 99024
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1188288 2:1062622272 3:1992416768 4:3187866624 5:4648972800
Statistics:
1 99024 D1=0.3559 D2=0.960184 D3+=1.54311
2 6376190 D1=0.699393 D2=1.0692 D3+=1.40085
3 27466090 D1=0.791496 D2=1.1291 D3+=1.38932
4 52510740 D1=0.862713 D2=1.2 D3+=1.41297
5 69780702 D1=0.756843 D2=1.34485 D3+=1.64054
Memory estimate for binary LM:
type      MB
probing 3176 assuming -p 1.5
probing 3671 assuming -r models -p 1.5
trie    1498 without quantization
trie     823 assuming -q 8 -b 8 quantization 
trie    1303 assuming -a 22 array pointer compression
trie     628 assuming -a 22 -q 8 -b 8 arra

In [ ]:
import kenlm
model = kenlm.Model('fivegram.arpa')

In [ ]:
from collections import Counter
def train_unigram(tokens, unigram_counts):
    for token in tokens:
        unigram_counts[token] += 1
    return unigram_counts

with open('/content/drive/MyDrive/com ling/contest 2/contest 2/train.src.tok', 'r') as f:
    unigram_counts = Counter()
    chunk_size = 50000
    tokens = []
    for i, line in enumerate(f):
        line_tokens = line.strip().split() # tokenize the line
        tokens.extend(line_tokens) # add tokens to list
        if len(tokens) > chunk_size:
            # process the chunk of tokens
            unigram_counts = train_unigram(tokens, unigram_counts)
            # reset the tokens list for the next chunk
            tokens = []
    # process the remaining tokens
    if len(tokens) > 0:
        unigram_counts = train_unigram(tokens, unigram_counts)


In [ ]:
len(unigram_counts)

99021

In [ ]:
first_letter = 'm'
context = 'the next'
def predict_word(context, vocab, first_letter, model_path):
    max_prob_so_far = float('-inf')
    best_word = ''
    for word in vocab:
        if word[0] == first_letter:
            sentence = context + ' ' + word
            prob = model.score(sentence)
            if prob > max_prob_so_far:
                best_word = word
                max_prob_so_far = prob
    return best_word

In [ ]:
first_letter = 'd'
context = 'china \' s deputy agriculture minister wei [UNK] is scheduled to arrive in taipei wednesday at the head of a six - member delegation for a six -'

In [ ]:
predict_word(context, unigram_counts, first_letter, model)

'major'

# Evaluate

In [ ]:
open('/content/drive/MyDrive/com ling/contest 2/contest 2/test_set_no_answer.csv', encoding='utf-8')

<_io.TextIOWrapper name='/content/drive/MyDrive/com ling/contest 2/contest 2/dev_set.csv' mode='r' encoding='utf-8'>

In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/com ling/contest 2/contest 2/test_set_no_answer.csv',encoding='utf-8')

In [ ]:
test

,context,first letter
0,ukraine and poland agreed to help each other i...,j
1,"richard blumenthal , the longtime attorney gen...",t
2,the leaders of france and britain say the coun...,t
3,""" some of the things i did in 1111 , i ' m bet...",n
4,"the white house , acknowledging that it was st...",","
...,...,...
94821,atlanta - three - fourths of the 111 georgia t...,a
94822,"thousands of aids experts , activists and poli...",s
94823,the number of u . s . workers applying for sta...,i
94824,whether toying with their opponent or just toy...,g


In [ ]:
test['tokens'] = test.context.apply(lambda x: x.split())
test_set_kenlm = test[['tokens', 'first letter']]
test_set_kenlm

,tokens,first letter
0,"[ukraine, and, poland, agreed, to, help, each,...",j
1,"[richard, blumenthal, ,, the, longtime, attorn...",t
2,"[the, leaders, of, france, and, britain, say, ...",t
3,"["", some, of, the, things, i, did, in, 1111, ,...",n
4,"[the, white, house, ,, acknowledging, that, it...",","
...,...,...
94821,"[atlanta, -, three, -, fourths, of, the, 111, ...",a
94822,"[thousands, of, aids, experts, ,, activists, a...",s
94823,"[the, number, of, u, ., s, ., workers, applyin...",i
94824,"[whether, toying, with, their, opponent, or, j...",g


In [ ]:
prediction = []
for i, row in test_set_kenlm.iterrows():
  rows = row['tokens']
  context = ' '.join(rows[-4:])
  possible_first_letter = row['first letter']
  next_word = predict_word(context, unigram_counts, possible_first_letter, model)
  prediction.append(next_word)
print(prediction[:100])

['join', 'tells', 'tense', 'night', ',', 'certain', 'consumers', 'the', 'in', 'country', 'he', 'losing', 'with', 'in', 'family', 'day', 'developments', 'the', 'other', 'qiongzhou', 'the', 'and', 'banning', 'insurgency', 'or', 'entire', '-', '111', 'a', 'of', 'available', 'latest', 'dutch', 'cares', 'downtown', 'western', 'resume', 'specifications', 'fix', 'in', '11111', 'the', 'say', 'and', "'", 'the', 'beef', 'day', 'billion', ',', 'seoul', 'of', 'authorities', 'secretary', 'old', 'from', 'one', 'and', 'rates', 'more', 'their', 'strikes', 'french', 'designed', 'and', 'local', 'june', 'a', ',', 'weeks', 'objections', 'had', 'more', '1111', 'some', 'deepening', 'zone', '11', 'china', 'some', 'irish', ',', 'johannesburg', '11', 'petrochemical', 'officer', 'new', 'southeastern', 'times', 'were', 'thursday', 'ongoing', ',', '-', '111', 'a', 'with', '[UNK]', '-', 'to']


In [ ]:
test_set_kenlm['prediction'] = prediction

In [ ]:
def accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1

    acc = correct / len(actual)
    return acc

In [ ]:
actual = test['answer']
predicted = test_set_kenlm['prediction']
accuracy(actual, predicted)

0.5405325599789085

In [ ]:
test_set_kenlm

,tokens,first letter,prediction
0,"[ukraine, and, poland, agreed, to, help, each,...",j,join
1,"[richard, blumenthal, ,, the, longtime, attorn...",t,tells
2,"[the, leaders, of, france, and, britain, say, ...",t,tense
3,"["", some, of, the, things, i, did, in, 1111, ,...",n,night
4,"[the, white, house, ,, acknowledging, that, it...",",",","
...,...,...,...
94821,"[atlanta, -, three, -, fourths, of, the, 111, ...",a,a
94822,"[thousands, of, aids, experts, ,, activists, a...",s,s
94823,"[the, number, of, u, ., s, ., workers, applyin...",i,in
94824,"[whether, toying, with, their, opponent, or, j...",g,give


In [ ]:
# export to csv
test_set_kenlm.to_csv('/content/sample_data/contest2_test_pred_kenlm.csv', index=None)

In [ ]:
open('/content/sample_data/contest2_test_pred_trigram.csv', encoding='utf-8')

<_io.TextIOWrapper name='/content/sample_data/contest2_test_pred_trigram.csv' mode='r' encoding='utf-8'>

In [ ]:
test_trigram = pd.read_csv('/content/contest2_test_pred_trigram.csv',encoding='utf-8')

In [ ]:
test_trigram

,tokens,first letter,prediction
0,"['ukraine', 'and', 'poland', 'agreed', 'to', '...",j,join
1,"['richard', 'blumenthal', ',', 'the', 'longtim...",t,NaN
2,"['the', 'leaders', 'of', 'france', 'and', 'bri...",t,treatment
3,"['""', 'some', 'of', 'the', 'things', 'i', 'did...",n,night
4,"['the', 'white', 'house', ',', 'acknowledging'...",",",","
...,...,...,...
94821,"['atlanta', '-', 'three', '-', 'fourths', 'of'...",a,a
94822,"['thousands', 'of', 'aids', 'experts', ',', 'a...",s,s
94823,"['the', 'number', 'of', 'u', '.', 's', '.', 'w...",i,in
94824,"['whether', 'toying', 'with', 'their', 'oppone...",g,give


In [ ]:
import random

nan_df = test_trigram[test_trigram['prediction'].isna()]
num = 0
for i, row in nan_df.iterrows():
    letter = row['first letter']
    words = {key: unigram_counts[key] for key in unigram_counts.keys() if key.startswith(letter)}
    candidates = Counter(words).most_common(5)
    random_word = random.choice(candidates)[0]
    test_trigram.at[i, 'prediction'] = random_word
    num+=1
print(num)


0


In [ ]:
actual = test['answer']
predicted = test_trigram['prediction']
accuracy(actual, predicted)

0.587208014764039

In [ ]:
# select rows in the first data frame where the value in the 'prediction' column is ''
selected_rows = test_trigram[test_trigram['prediction'].isna()]

# extract corresponding rows from the second data frame using the index values
matching_rows = test_set_kenlm.loc[selected_rows.index]

# replace values in the 'prediction' column of the selected rows in the first data frame with the corresponding values from the second data frame
test_trigram.loc[selected_rows.index, 'prediction'] = matching_rows['prediction'].values


In [ ]:
test_trigram

,tokens,first letter,prediction
0,"['ukraine', 'and', 'poland', 'agreed', 'to', '...",j,join
1,"['richard', 'blumenthal', ',', 'the', 'longtim...",t,tells
2,"['the', 'leaders', 'of', 'france', 'and', 'bri...",t,treatment
3,"['""', 'some', 'of', 'the', 'things', 'i', 'did...",n,night
4,"['the', 'white', 'house', ',', 'acknowledging'...",",",","
...,...,...,...
94821,"['atlanta', '-', 'three', '-', 'fourths', 'of'...",a,a
94822,"['thousands', 'of', 'aids', 'experts', ',', 'a...",s,s
94823,"['the', 'number', 'of', 'u', '.', 's', '.', 'w...",i,in
94824,"['whether', 'toying', 'with', 'their', 'oppone...",g,give


In [ ]:
import random
nan_df = test_set_kenlm[test_set_kenlm['prediction'].isna()]
num = 0
for i, row in nan_df.iterrows():
    letter = row['first letter']
    words = {key: unigram_counts[key] for key in unigram_counts.keys() if key.startswith(letter)}
    random_word = Counter(words).most_common()
    #random_word = random.choice(candidates)[0]
    test_set_kenlm.at[i, 'prediction'] = random_word
    num+=1
print(num)

0


In [ ]:
actual = test['answer']
predicted = test_set_kenlm['prediction']
accuracy(actual, predicted)

0.5405325599789085

In [ ]:
test_result = test_trigram['prediction']

In [ ]:
test_result

0             join
1            tells
2        treatment
3            night
4                ,
           ...    
94821            a
94822            s
94823           in
94824         give
94825    milestone
Name: prediction, Length: 94826, dtype: object

In [ ]:
test_result.to_csv('/content/sample_data/test_set_pred.txt', index=None)

In [ ]:
# export to csv
test_trigram.to_csv('/content/sample_data/contest2_test_pred_combine_ngram.csv', index=None)